In [1]:
import numpy as np
import os
from keras.models import Model
from keras.layers import Dense, Embedding, Activation, Permute
from keras.layers import Input, Flatten, Dropout
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed, Bidirectional

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from custom_recurrents import AttentionDecoder
from tdd import _time_distributed_dense

In [3]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [4]:
from cc_train_val_test import get_upsampled_raw_dataset

In [5]:
import pandas as pd

In [6]:
upsampled_raw_dataset = get_upsampled_raw_dataset()

In [7]:
t,v,te = upsampled_raw_dataset

In [8]:
t=t.dropna()
v=v.dropna()
te=te.dropna()

In [9]:
tclue = t.clue.apply(lambda x:list(x))
vclue = v.clue.apply(lambda x:list(x))
teclue = te.clue.apply(lambda x:list(x))

In [10]:
tokenizer = Tokenizer(filters='')

In [11]:
tokenizer.fit_on_texts(pd.concat([tclue,vclue,teclue]))

In [12]:
tx = pad_sequences(tokenizer.texts_to_sequences(tclue),maxlen=50)
vx = pad_sequences(tokenizer.texts_to_sequences(vclue),maxlen=50)
tex = pad_sequences(tokenizer.texts_to_sequences(teclue),maxlen=50)

In [13]:
ty = t[t.columns[4:]]
vy = v[v.columns[4:]]
tey = te[te.columns[4:]]

In [14]:
tey

,is_anagram,is_homophone,is_double,is_cryptic,is_contain,is_reverse,is_alternate,is_init,is_delete,is_hidden,is_spoonerism,is_palindrome
78171,True,False,False,False,False,False,False,False,False,False,False,False
78182,True,False,False,False,False,False,False,False,False,False,False,False
78184,True,False,False,False,False,False,False,False,False,False,False,False
78191,True,False,False,False,False,False,False,False,False,False,False,False
78202,True,False,False,False,False,False,False,False,False,False,False,False
78215,True,False,False,False,False,False,False,False,False,False,False,False
78216,True,False,False,False,False,False,False,False,False,False,False,False
78230,True,False,False,False,False,False,False,False,False,False,False,False
78235,True,False,False,False,False,False,False,False,False,False,False,False
78247,True,False,False,False,False,False,False,False,False,False,False,False


In [19]:
pad_length = 50
n_chars = len(tokenizer.index_word)+1
n_labels = 12
encoder_units=256,
decoder_units=256,
trainable=True
return_probabilities=True
embedding_learnable=False

In [20]:
tx.shape

(177792, 50)

In [21]:
def simpleNMT(pad_length = 50,n_chars = len(tokenizer.index_word)+1,n_labels = 12,encoder_units=256,decoder_units=256,trainable=True,return_probabilities=True):
    """
    Builds a Neural Machine Translator that has alignment attention
    :param pad_length: the size of the input sequence
    :param n_chars: the number of characters in the vocabulary
    :param n_labels: the number of possible labelings for each character
    :param embedding_learnable: decides if the one hot embedding should be refinable.
    :return: keras.models.Model that can be compiled and fit'ed

    *** REFERENCES ***
    Lee, Jason, Kyunghyun Cho, and Thomas Hofmann. 
    "Neural Machine Translation By Jointly Learning To Align and Translate" 
    """
    input_ = Input(shape=(pad_length,), dtype='float32')
    input_embed = Embedding(n_chars, n_chars,
                            input_length=pad_length,
                            trainable=embedding_learnable,
                            weights=[np.eye(n_chars)],
                            name='OneHot')(input_)

    rnn_encoded = Bidirectional(LSTM(encoder_units, return_sequences=True),
                                name='bidirectional_1',
                                merge_mode='concat',
                                trainable=trainable)(input_embed)

    atten_decoder = AttentionDecoder(decoder_units,
                             name='attention_decoder_1',
                             output_dim=n_labels,
                             return_probabilities=return_probabilities,
                             trainable=trainable)(rnn_encoded)
    y_hat = Flatten()(atten_decoder)
    y_hat = Dense(12,activation='softmax')(y_hat)

    model = Model(inputs=input_, outputs=y_hat)

    return model

In [22]:
model = simpleNMT()

inputs shape: (?, ?, 512)


In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 50)                0         
_________________________________________________________________
OneHot (Embedding)           (None, 50, 73)            5329      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 512)           675840    
_________________________________________________________________
attention_decoder_1 (Attenti (None, 50, 50)            937372    
_________________________________________________________________
flatten_1 (Flatten)          (None, 2500)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 12)                30012     
Total params: 1,648,553
Trainable params: 1,643,224
Non-trainable params: 5,329
______________________________________________________________

In [ ]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint
import os
cp = ModelCheckpoint("./weights/NMT.{epoch:02d}-{val_loss:.2f}.hdf5",
                     monitor='val_loss',
                     verbose=0,
                     save_best_only=True,
                     save_weights_only=True,
                     mode='auto')

# create a directory if it doesn't already exist
if not os.path.exists('./weights'):
    os.makedirs('./weights/')

In [ ]:
model.fit(tx,ty,validation_data=(vx,vy),batch_size=32,epochs=20,callbacks=[cp])

In [ ]:
print(embedding_learnable)